In [2]:
import pandas as pd
import random
import numpy as np

In [3]:
import lightgbm
import pandas as pd
from utils.helper_functions import get_data
from utils.background_data import BackgroundData
import numpy as np
import argparse
from pathlib import Path


file_name = "model_MQ2008"
dataset = "MQ2008"
model_type = "listwise"
background_samples = 100

/opt/miniconda3/envs/rankingSHAP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

def train_model(
    train_data, eval_data, ranking_model_type="listwise", save_to_file="model.txt"
):
    assert ranking_model_type in ["listwise", "pairwise", "pointwise"]

    TX, Ty, Tqids = train_data
    VX, Vy, Vqids = eval_data

    # create pandas dataframe of qids to easily construct group list
    T_df = pd.DataFrame(data=Tqids, columns=["Tqids"])
    V_df = pd.DataFrame(data=Vqids, columns=["Vqids"])
    qid_count_list_train = T_df.groupby("Tqids")["Tqids"].count().to_numpy()
    qid_count_list_val = V_df.groupby("Vqids")["Vqids"].count().to_numpy()

    # model training
    if ranking_model_type == "listwise":
        # train lambda rank model
        model = lightgbm.LGBMRanker(
            objective="lambdarank",
            metric="ndcg",
        )

        model.fit(
            X=TX,
            y=Ty,
            group=qid_count_list_train,
            eval_set=[(VX, Vy)],
            eval_group=[qid_count_list_val],
            eval_at=10,
        )

    if ranking_model_type == "pairwise":
        # train lambda rank model
        model = lightgbm.LGBMRanker(
            objective="binary",
            metric="ndcg",
        )

        model.fit(
            X=TX,
            y=Ty,
            group=qid_count_list_train,
            eval_set=[(VX, Vy)],
            eval_group=[qid_count_list_val],
            eval_at=10,
        )

    if ranking_model_type == "pointwise":
        # train lambda rank model
        model = lightgbm.LGBMRanker(
            objective="regression",
            metric="ndcg",
        )

        model.fit(
            X=TX,
            y=Ty,
            group=qid_count_list_train,
            eval_set=[(VX, Vy)],
            eval_group=[qid_count_list_val],
            eval_at=10,
        )
    model.booster_.save_model(save_to_file)
    return model



In [5]:
from pyltr.data.letor import read_dataset
def get_data(data_file):
    # print("Loading data from:", data_file)
    with open(data_file) as f:
        EX, Ey, Eqids, _ = read_dataset(f)
    return EX, Ey, Eqids

In [33]:

# Get train, eval_data
data_directory = Path("/Users/keqiaoli/Desktop/RankingSHAP/RankingSHAP/data/" + dataset + "/Fold1/")


# train_data = get_data(data_file=data_directory /"train.txt")
train_data = get_data(data_file=data_directory / "train.txt")
test_data = get_data(data_file=data_directory / "test.txt")
eval_data = get_data(data_file=data_directory / "vali.txt")


In [34]:
test_data_pd = pd.DataFrame(test_data[0])

In [40]:
#convert test data to pandas dataframe
test_data_pd = pd.DataFrame(test_data[0])
test_data_pd["relevance"] = test_data[1]
test_data_pd["qid"] = test_data[2]

In [42]:
test_data_pd['qid'].value_counts()

qid
19353    119
18574    117
19116    115
19782    114
18230     61
        ... 
18489      7
19511      7
19186      7
19997      7
18889      6
Name: count, Length: 156, dtype: int64

In [8]:
# output train_data to check

eval_data = get_data(data_file=data_directory / "vali.txt")



In [10]:
model = train_model(
    train_data=train_data,
    eval_data=eval_data,
    ranking_model_type=model_type,
    save_to_file=Path("results/model_files/" + file_name),
)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002285 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9233
[LightGBM] [Info] Number of data points in the train set: 9630, number of used features: 40


In [75]:
import shap


class BackgroundData:
    def __init__(self, data, summarization_type=None, summary_length=1):
        self.data = data
        self.summarization_type = summarization_type
        self.summary_length = summary_length
        self.background_summary = self.get_background_summary()

    def get_background_summary(self):
        if self.summarization_type is None:
            background_summary = self.data
        elif self.summarization_type == "random_sample":

            background_summary = shap.sample(self.data, self.summary_length, random_state=42)
        else:
            NotImplementedError
        return background_summary

In [76]:
import numpy as np
import shap

# Step 1: Set global NumPy seed
# np.random.seed(42)

# Step 2: NumPy sampling using the global RNG
rng = np.random.default_rng(42)
indices = rng.permutation(len(train_data[0]))[:2]
sampled_rows = train_data[0][indices]
print("NumPy sample:")
print(sampled_rows)

# Step 3: SHAP sampling (uses global np.random by default)
# np.random.seed(42)  # Reset seed again to ensure same sampling order
background_data = BackgroundData(
    train_data[0], summarization_type="random_sample", summary_length=2
)
print("SHAP sample:")
print(background_data.background_summary)


NumPy sample:
[[0.011491 0.       0.       0.       0.010859 0.       0.       0.
  0.       0.       0.161037 0.       0.       0.       0.16279  0.009547
  0.23913  0.04     0.25     0.009766 0.165269 0.188372 0.098929 0.291973
  0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.153157 0.185041 0.096407 0.286964
  0.5      0.267442 0.       0.004206 0.010949 0.005319]
 [0.064132 0.       0.       0.       0.063703 0.       0.       0.
  0.       0.       0.       0.       0.       0.       0.       0.074947
  0.111111 0.       1.       0.074958 0.311845 0.601238 0.522513 0.583854
  0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.31541  0.601227 0.522273 0.589928
  1.       1.       0.       0.026507 0.020619 0.      ]]
SHAP sample:
[[4.00000e-02 2.50000e-01 5.00000e-01 0.00000e+00 4.25000e-02 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.53090e-02 2.5000

In [77]:
sampled_rows.shape

(2, 46)

In [78]:
background_data.background_summary.shape

(2, 46)

In [57]:
# Prepare background data training:
np.random.seed(42)
background_data = BackgroundData(
    train_data[0], summarization_type="random_sample", summary_length=2
)

In [58]:
background_data.background_summary

array([[1.54963e-01, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.54795e-01,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        6.43906e-01, 0.00000e+00, 0.00000e+00, 0.00000e+00, 6.48643e-01,
        6.61950e-01, 2.43900e-02, 5.41667e-01, 6.25000e-02, 6.62106e-01,
        2.64384e-01, 4.15081e-01, 4.20185e-01, 4.26565e-01, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 2.65000e-01, 4.17481e-01, 4.20382e-01, 4.42983e-01,
        1.25000e-01, 1.09756e-01, 0.00000e+00, 4.05000e-04, 1.21950e-02,
        0.00000e+00],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 3.62300e-03,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 3.91300e-03,
        7.80000e-05, 0.00000e+00, 5.33333e-01, 0.00000e+00, 0.00000e+00,
        4.88693e-01, 5.90138e

In [59]:
# Prepare background data training:
background_data = BackgroundData(
    train_data[0], summarization_type="random_sample", summary_length=2
)
background_data.background_summary

array([[1.54963e-01, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.54795e-01,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        6.43906e-01, 0.00000e+00, 0.00000e+00, 0.00000e+00, 6.48643e-01,
        6.61950e-01, 2.43900e-02, 5.41667e-01, 6.25000e-02, 6.62106e-01,
        2.64384e-01, 4.15081e-01, 4.20185e-01, 4.26565e-01, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 2.65000e-01, 4.17481e-01, 4.20382e-01, 4.42983e-01,
        1.25000e-01, 1.09756e-01, 0.00000e+00, 4.05000e-04, 1.21950e-02,
        0.00000e+00],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 3.62300e-03,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 3.91300e-03,
        7.80000e-05, 0.00000e+00, 5.33333e-01, 0.00000e+00, 0.00000e+00,
        4.88693e-01, 5.90138e

In [63]:
np.random.seed(42)
sample_indices = np.random.choice(train_data[0].shape[0], size=2, replace=False)
sampled_rows = train_data[0][sample_indices]

In [64]:
sampled_rows

array([[4.00000e-02, 2.50000e-01, 5.00000e-01, 0.00000e+00, 4.25000e-02,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        1.53090e-02, 2.50000e-01, 5.13927e-01, 0.00000e+00, 1.98360e-02,
        1.41670e-02, 6.66670e-02, 1.42857e-01, 5.23810e-01, 1.43350e-02,
        6.54987e-01, 7.38133e-01, 5.69881e-01, 7.70340e-01, 3.83214e-01,
        4.36213e-01, 5.61904e-01, 3.38645e-01, 4.57523e-01, 2.08700e-01,
        1.85880e-01, 1.55677e-01, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 6.97249e-01, 7.40984e-01, 6.07830e-01, 7.70739e-01,
        3.33333e-01, 4.83871e-01, 0.00000e+00, 6.90000e-05, 1.75100e-03,
        4.74359e-01],
       [1.29870e-01, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.29870e-01,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        1.15365e-01, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.15338e-01,
        1.16210e-02, 1.81818e-01, 1.00000e+00, 3.33330e-02, 9.06900e-03,
        1.00000e+00, 9.51584e

In [ ]:
# Prepare background data training:
background_data = BackgroundData(
    train_data[0], summarization_type="random_sample", summary_length=background_samples
)
np.save(
    Path("results/background_data_files/train_background_data_" + dataset + ".npy"),
    background_data.background_summary,)

In [26]:
import numpy as np

# helper function, rank list with highest rank for highest value
import pandas as pd
from pyltr.data.letor import read_dataset

def get_data(data_file):
    with open(data_file) as evalfile:
        EX, Ey, Eqids, _ = read_dataset(evalfile)
    return EX, Ey, Eqids

from pathlib import Path
dataset = "MQ2008"
data_directory = Path("/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/data/" + dataset + "/Fold1/")
train_data = get_data(data_file=data_directory / "train.txt")




In [17]:
background_data.background_summary

array([[1.54963e-01, 0.00000e+00, 0.00000e+00, ..., 4.05000e-04,
        1.21950e-02, 0.00000e+00],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [4.39210e-01, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [2.94290e-02, 6.66667e-01, 5.00000e-01, ..., 5.26308e-01,
        1.05260e-02, 1.00000e+00],
       [3.17919e-01, 0.00000e+00, 0.00000e+00, ..., 5.51200e-03,
        1.02190e-01, 2.56410e-02],
       [6.36900e-03, 1.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        1.00000e+00, 8.63830e-01]], shape=(100, 46))

# Designing candidates from the search queries 

We design a set of candidates that will help us investige different ranking scenarios. 
- non-qualified-privileged: Candidate that does not meet the basic job requirements but comes from the priviledged nepotism-university
- non-qualified: Candidate that does not meet the basic job requirements, from a non-previledged us-university
- qualified-1, qualified-2, qualified-3: Candidates that meet the basic job requirements fromm us-university
- good-graduate: A candidate without any job experience but with good portfolio otherwise 
- qualified-biased: Candidate that does not meet the basic job requirements, from the bias-university, which the model is biased against
- qualified-net: Candidate that does not meet the basic job requirements, from a ger-university
- qualified-ger: Candidate that does not meet the basic job requirements, from a net-university


In [24]:
candidates = {
    "non-qualified-privileged" : [0.8, 0.6, 3.6, "nepotism-university", False],
    "non-qualified" : [0.7, 0.7, 3.2, "us-university", False],
    "qualified-1" : [0.8, 0.55, 3.5, "us-university", True],
    "qualified-2" : [0.7, 0.3, 3, "us-university", True],
    "qualified-3" : [0.9, 0.8, 3, "us-university", True],
    "good-graduate": [0, 0.9, 4, "us-university", True],
    "qualified-biased" : [0.8, 0.6, 3.6, "bias-university", True],
    "qualified-net": [0.7, 0.9, 8, "net-university", True],
    "qualified-ger": [0.8, 0.8, 1, "ger-university", True],
}

We transform the university fearture into a numeric feature to avoid mixed type arrays

In [25]:
numerical_university_values = {
    "ger-university": -1 , "net-university": 10, "us-university": 2, 
    "nepotism-university": 1, "bias-university": 3
    } 

def university_to_numerical(candidate): 
    candidate[features["university"]] =  numerical_university_values[candidate[features["university"]]]
    return candidate
candidates = {c: university_to_numerical(f) for c, f in candidates.items()}
candidates

{'non-qualified-privileged': [0.8, 0.6, 3.6, 1, False],
 'non-qualified': [0.7, 0.7, 3.2, 2, False],
 'qualified-1': [0.8, 0.55, 3.5, 2, True],
 'qualified-2': [0.7, 0.3, 3, 2, True],
 'qualified-3': [0.9, 0.8, 3, 2, True],
 'good-graduate': [0, 0.9, 4, 2, True],
 'qualified-biased': [0.8, 0.6, 3.6, 3, True],
 'qualified-net': [0.7, 0.9, 8, 10, True],
 'qualified-ger': [0.8, 0.8, 1, -1, True]}

In [26]:
grade_range = {numerical_university_values[university]: r for university, r in grade_range.items()}

# Sampling a set of background candidates

Since features can not be simply removed from the input to estimate the impact that each feature has on the different coalitions, a set of background candidates is being sampled that is used to mask the features that need to be hidden instead. Here we sample a set of 100 candidates that are randomly. All features are sampled uniforly within the corresponding range, assuming that the features are uncorrelated with each other. The grade feature is sampled dependent on the university that defines the range of the grades. 

In [27]:
def sample_candidates(samples):
    # a = ["prior_experience", "skills", "grades", "university", "meets_qualifications", "h-index", "job_title"]
    candidates = []

    for i in range(samples):
        candidate = {feature: random.choice(feature_range[feature]) for feature in ["university", "requirements"]}
        for feature in ["experience", "skills"]:
            candidate[feature] = random.uniform(feature_range[feature][0], feature_range[feature][1])
        if candidate["university"] in ["us-university", "nepotism-university", "bias-university"]:
            candidate["grade"] = random.uniform(1, 4)
        elif candidate["university"] == "ger-university":
            candidate["grade"] = random.uniform(1, 4)
        elif candidate["university"] == "net-university":
            candidate["grade"] = random.uniform(6, 10)
        candidates.append(candidate)
    return candidates


We sample 100 candidates.

In [28]:
background_samples = sample_candidates(100)
background_samples = pd.DataFrame(background_samples)
background_samples = background_samples[features.keys()]
background_samples = np.array(background_samples)
background_samples = np.array([university_to_numerical(b) for b in background_samples])
background_samples[:2]

array([[0.013061410628058101, 0.6216656810946519, 2.0821460111592853, 3,
        False],
       [0.14680107283639943, 0.710773904086238, 3.3284405590714834, 3,
        True]], dtype=object)

# Designing the query scenarios

We design the following scenarios to evaluate the different explanation approaches on: 

In [29]:
query_candidates = {
    "average" : ["non-qualified",  "qualified-1", "qualified-2"],
    "nepotism" : ["non-qualified-privileged", "non-qualified", "qualified-1", "qualified-2"],
    "qualified" : ["qualified-1", "qualified-2", "qualified-3"],
    "international": ["qualified-net","qualified-ger", "qualified-3", "non-qualified"],
    "biased": ["qualified-biased", "non-qualified", "qualified-1", "qualified-2"],
    "good-graduate": ["qualified-3",  "qualified-1", "qualified-2", "good-graduate"]
    # "queries_random" : [sample_candidates(10) for _ in range(100)]
}

In [30]:
queries = {query: [candidates[candidate_name] for candidate_name in candidate_names] for query, candidate_names in query_candidates.items()}
queries

{'average': [[0.7, 0.7, 3.2, 2, False],
  [0.8, 0.55, 3.5, 2, True],
  [0.7, 0.3, 3, 2, True]],
 'nepotism': [[0.8, 0.6, 3.6, 1, False],
  [0.7, 0.7, 3.2, 2, False],
  [0.8, 0.55, 3.5, 2, True],
  [0.7, 0.3, 3, 2, True]],
 'qualified': [[0.8, 0.55, 3.5, 2, True],
  [0.7, 0.3, 3, 2, True],
  [0.9, 0.8, 3, 2, True]],
 'international': [[0.7, 0.9, 8, 10, True],
  [0.8, 0.8, 1, -1, True],
  [0.9, 0.8, 3, 2, True],
  [0.7, 0.7, 3.2, 2, False]],
 'biased': [[0.8, 0.6, 3.6, 3, True],
  [0.7, 0.7, 3.2, 2, False],
  [0.8, 0.55, 3.5, 2, True],
  [0.7, 0.3, 3, 2, True]],
 'good-graduate': [[0.9, 0.8, 3, 2, True],
  [0.8, 0.55, 3.5, 2, True],
  [0.7, 0.3, 3, 2, True],
  [0, 0.9, 4, 2, True]]}

# The ranking model

![missing model png file](data/Images/biased_unbiased_models_zoomed_in.png) 

In [31]:
def ranking_model(feature_vectors, biased=True): #feature_vector must be a np.array
    # def get_numerical_features(x):
    #     return [x[features[feature_name]] for feature_name in ["prior_experience", "skills", "grades","h-index"]]
    ranking_scores = []
    single_input = False
    if len(np.array(feature_vectors).shape) == 1 or np.array(feature_vectors).shape[1] == 0:
        feature_vectors = [feature_vectors]
        single_input = True
    for feature_vector in feature_vectors:
        ranking_score = feature_vector[features["skills"]] + feature_vector[features["experience"]]
        normalized_grade = feature_vector[features["grade"]]
        
        # Add a bias, normalize grades based on system.
        g_range = grade_range[feature_vector[features["university"]]]
        if g_range[0] < g_range[1]:
            normalized_grade = (g_range[1] - normalized_grade)/ (g_range[1]-g_range[0])
        else:
            normalized_grade = (normalized_grade - g_range[1])/ (g_range[0]-g_range[1])
        ranking_score += normalized_grade
    
        if biased and feature_vector[features["university"]] == 3:
            ranking_score = ranking_score * 0.7

        if not feature_vector[features["requirements"]] == 1:
            if not (feature_vector[features["university"]] == 1 and biased):
                ranking_score = ranking_score * 0.1
        ranking_scores.append(ranking_score)
    if single_input:
        return ranking_scores[0]
    return np.array(ranking_scores)


# Generating explanations for each scenario 

We generate explanations for the query scenarios. We investigate the biased model first. For investigating the unbiased model instead, uncomment the 7. line in the code. 

In [13]:
from approaches.ranking_shap import RankingShap
from approaches.greedy_listwise import GreedyListwise
from approaches.pointwise_shap import AggregatedShap
from scipy.stats import kendalltau
from approaches.ranking_lime import RankingLIME
from functools import partial 

ranking_model_ = partial(ranking_model, biased=True) 



rank_similarity_coefficient = lambda x,y: kendalltau(x,y)[0]
explanation_size = 5

ranking_shap_explainer = RankingShap(
    permutation_sampler="kernel",
    background_data=background_samples,
    original_model=ranking_model_,
    explanation_size=num_features,
    name="rankingshap_no_weighting",
    rank_similarity_coefficient=rank_similarity_coefficient
)

# While in the paper we use the greedy feature selection baseline to compare our approach with, 
# here we design a variant of this approach that naively adds the marginal contribution that each 
# feature adds to the current explanation set for the greedily added features. The feature selection
# baseline can be attained through binarization of these values. 

greedy_explainer_iter = GreedyListwise(
    background_data=background_samples,
    model=ranking_model_,
    explanation_size=2,
    name="greedy_listwise_iter",
    feature_attribution_method="iter",
    mixed_type_input=True
)


aggregated_shap_explainer = AggregatedShap(
    background_data=background_samples,
    model=ranking_model_,
    explanation_size=num_features,
    name="aggregated_shap_top_5",
    aggregate_over_top=5,
)

explainers = [ranking_shap_explainer, greedy_explainer_iter, aggregated_shap_explainer]


In [14]:
explanation_dfs = {}
for explainer in explainers:
    explanations = {}
    for current_query in queries:
        features_selection, feature_attribution = explainer.get_query_explanation(query_features=queries[current_query], query_id=current_query)
        # features_selection, feature_attribution = explainer.get_query_explanation(query_features=np.array(queries[current_query]), query_id=current_query)
        rankingscores = [ranking_model(candidate) for candidate in queries[current_query]]
        explanations[current_query] = feature_attribution
    results = pd.DataFrame({ex_name: {expl[0]: expl[1] for expl in ex.explanation} for ex_name, ex  in explanations.items()}).sort_index()
    results["features"] = list(features.keys())
    results = results.set_index("features")
    explanation_dfs[explainer.name]  = results


100%|██████████| 1/1 [00:00<00:00,  2.94it/s]


In [15]:
explanation_dfs = pd.concat(explanation_dfs, names=['approach'])

In [16]:
import matplotlib.pyplot as plt
for column in explanation_dfs.columns: 
    e = explanation_dfs[column].unstack(level=0)
    e.plot(kind='barh', color=['red', 'green', 'blue'])
    plt.xlim(-1, 1)
    plt.title(column)
    plt.show()

In [1]:
import  numpy as np

# helper function, rank list with highest rank for highest value
import pandas as pd
from pyltr.data.letor import read_dataset

with open("/Users/keqiaoli/Desktop/RankingSHAP/RankingSHAP/data/MQ2008/Fold1/train.txt") as evalfile:

    EX, Ey, Eqids, _ = read_dataset(evalfile)



In [2]:
EX

array([[0.007477, 0.      , 1.      , ..., 0.      , 0.      , 0.007042],
       [0.603738, 0.      , 1.      , ..., 0.003708, 0.333333, 1.      ],
       [0.214953, 0.      , 0.      , ..., 1.      , 1.      , 0.021127],
       ...,
       [1.      , 0.      , 0.      , ..., 0.060915, 0.454545, 0.      ],
       [0.259641, 0.6     , 0.      , ..., 0.051975, 0.090909, 0.      ],
       [0.791031, 0.      , 0.      , ..., 0.001754, 0.181818, 0.      ]],
      shape=(9630, 46))

In [3]:
Ey

array([0., 0., 0., ..., 0., 0., 0.], shape=(9630,))

In [4]:
Eqids

array(['10002', '10002', '10002', ..., '15925', '15925', '15925'],
      shape=(9630,), dtype='<U5')